In [4]:
import torch
from torch import tensor
from torch import nn

In [28]:
def cross2d(X,Y):
    hight=Y.shape[0]
    width=Y.shape[1]
    out_h=X.shape[0]-hight+1
    out_w=X.shape[1]-width+1
    out  = torch.zeros(size=(out_h,out_w),dtype=torch.float)
    for i in range(out_h):
        for j in range(out_w):
            out[i,j]=(X[i:i+hight,j:j+width]*Y).sum()
    return out

In [43]:
X=torch.ones(size=(6,8))
X[:,2:6]=0
K=torch.tensor([1,-1]).reshape(1,-1)
Y=cross2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [34]:
#define convolution
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(self.__init__(kernel_size))
        self.weight=nn.Parameter(torch.rand(kernel_size))
        self.bias=nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        return cross2d(x,self.weight)+self.bias
        

In [31]:
?nn.Module.zero_grad

Signature: nn.Module.zero_grad(self, set_to_none: bool = False) -> None
Docstring:
Sets gradients of all model parameters to zero. See similar function
under :class:`torch.optim.Optimizer` for more context.

Args:
    set_to_none (bool): instead of setting to zero, set the grads to None.
        See :meth:`torch.optim.Optimizer.zero_grad` for details.
File:      d:\anaconda\lib\site-packages\torch\nn\modules\module.py
Type:      function


In [46]:
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)

X=X.reshape((1,1,6,-1))
Y=Y.reshape((1,1,6,-1))
lamb=0.01

for i in range(50):
    Y_hat=conv2d(X)
    l=(Y-Y_hat)**2
    conv2d.zero_grad()
    l.sum().backward()
    
    conv2d.weight.data[:] -= lamb*conv2d.weight.grad
    if(i%5==0):
        print(l.sum())

tensor(19.2946, grad_fn=<SumBackward0>)
tensor(3.4450, grad_fn=<SumBackward0>)
tensor(0.9592, grad_fn=<SumBackward0>)
tensor(0.2672, grad_fn=<SumBackward0>)
tensor(0.0744, grad_fn=<SumBackward0>)
tensor(0.0207, grad_fn=<SumBackward0>)
tensor(0.0058, grad_fn=<SumBackward0>)
tensor(0.0016, grad_fn=<SumBackward0>)
tensor(0.0004, grad_fn=<SumBackward0>)
tensor(0.0001, grad_fn=<SumBackward0>)


In [51]:
from d2l import torch as d2l
import torch

def corr2d_multi(X,K):
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K))

In [53]:
X=torch.ones(size=(2,3,4))
K=torch.ones(size=(2,1,3))
print(corr2d_multi(X,K))

tensor([[6., 6.],
        [6., 6.],
        [6., 6.]])


In [54]:
def corr2d_multi_in_out(X,K):
    return torch.stack([corr2d_multi(X,k) for k in K],0)
K=torch.stack([K,K+1,K+2],0)
print(corr2d_multi_in_out(X,K))

tensor([[[ 6.,  6.],
         [ 6.,  6.],
         [ 6.,  6.]],

        [[12., 12.],
         [12., 12.],
         [12., 12.]],

        [[18., 18.],
         [18., 18.],
         [18., 18.]]])


In [58]:
def corr2d_in_out_1x1(X,K):
    c_i,h,w= X.shape
    c_o=K.shape[0]
    X=X.reshape((c_i,h*w))
    K=K.reshape((c_o,c_i)) #cause this is 1X1!
    Y=torch.matmul(K,X)
    return Y.reshape((c_o,h,w))
    

In [59]:
K=torch.ones(size=(2,3,1,1))
X=torch.ones(size=(3,3,3))
Y=corr2d_in_out_1x1(X,K)
Y

tensor([[[3., 3., 3.],
         [3., 3., 3.],
         [3., 3., 3.]],

        [[3., 3., 3.],
         [3., 3., 3.],
         [3., 3., 3.]]])

In [6]:
def pool2d(X,pool_h,pool_w):
    h,w= X.shape
    out=torch.zeros(size=(h-pool_h+1,w-pool_w+1))
    for i in range(h-pool_h+1):
        for j in range(w-pool_w+1):
            out[i,j]=X[i:i+pool_h,j:j+pool_w].max()
    return out
            

In [10]:
X=torch.normal(0,1,size=(5,5))
out=pool2d(X,2,2)
print(out)
print(X)

tensor([[ 1.2360,  1.2360,  0.7705,  1.4596],
        [ 0.4213,  0.6769,  0.8251,  1.4596],
        [-0.2091,  0.6769,  0.8251,  0.8251],
        [-0.2091,  0.5615,  0.5615,  0.5960]])
tensor([[-0.1414,  1.2360, -1.1413,  0.7705, -0.7763],
        [ 0.4213, -0.1318, -0.9049, -1.0760,  1.4596],
        [-0.3071, -0.5148,  0.6769,  0.8251, -0.5902],
        [-0.2091, -0.4490,  0.5615, -1.5272,  0.5960],
        [-0.5863, -1.1539, -1.2078, -0.6966,  0.0230]])
